---
jupytext:
  cell_metadata_filter: all, -hidden, -heading_collapsed, -run_control, -trusted
  notebook_metadata_filter: all, -jupytext.text_representation.jupytext_version, -jupytext.text_representation.format_version, -language_info.version, -language_info.codemirror_mode.version, -language_info.codemirror_mode, -language_info.file_extension, -language_info.mimetype, -toc
  text_representation:
    extension: .md
    format_name: myst
kernelspec:
  display_name: Python 3 (ipykernel)
  language: python
  name: python3
language_info:
  name: python
  nbconvert_exporter: python
  pygments_lexer: ipython3
nbhosting:
  title: 'Correction du TP choix de variables'
  version: '1.0'
---

<div class="licence">
<span><img src="media/logo_IPParis.png" /></span>
<span>Lisa BEDIN<br />Pierre André CORNILLON<br />Eric MATZNER-LOBER</span>
<span>Licence CC BY-NC-ND</span>
</div>

# Modules

Importer les modules pandas (comme `pd`) numpy (commme `np`) matplotlib.pyplot (comme `plt`) et statsmodels.formula.api (comme `smf`).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Régression ridge sur les données d'ozone


## Importation des données

Importer les données d'ozone `ozonecomplet.csv` (dans Fun Campus les données sont dans le répertoire `data/`) et éliminer les deux dernières variables (qualitatives) et faites un résumé numérique par variable \[méthode `astype` sur la colonne du DataFrame et méthode `describe` sur l'instance DataFrame\]

In [2]:
ozone = pd.read_csv("data/ozonecomplet.csv", header=0, sep=";")
ozone = ozone.drop(['nomligne', 'Ne', 'Dv'], axis=1)
ozone.describe()

,O3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,O3v
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,90.303571,18.360714,21.526786,22.627679,4.928571,5.017857,4.830357,-1.214346,-1.611004,-1.690683,90.571429
std,28.187225,3.122726,4.042321,4.530859,2.594916,2.281860,2.332259,2.632742,2.795673,2.810198,28.276853
min,42.000000,11.300000,14.000000,14.900000,0.000000,0.000000,0.000000,-7.878500,-7.878500,-9.000000,42.000000
25%,70.750000,16.200000,18.600000,19.275000,3.000000,4.000000,3.000000,-3.276450,-3.564700,-3.939200,71.000000
50%,81.500000,17.800000,20.550000,22.050000,6.000000,5.000000,5.000000,-0.866000,-1.879400,-1.549650,82.500000
75%,106.000000,19.925000,23.550000,25.400000,7.000000,7.000000,7.000000,0.694600,0.000000,0.000000,106.000000
max,166.000000,27.000000,33.500000,35.500000,8.000000,8.000000,8.000000,5.196200,6.577800,5.000000,166.000000


## Sélection descendante/backward

Proposer une fonction qui permet la sélection descendante/backward. Elle utilisera les formules de `statsmodels` et incluera toujours la constante. En entrée serviront trois arguments: le DataFrame des données, la formule de départ et le critère (AIC ou BIC). La fonction retournera le modèle estimé via `smf.ols`

La fonction commence avec le modèle complet.

-   Nous séparons la variable réponse (objet `response`) des variables explicatives,
-   Ces dernières sont transformées en un ensemble (objet `start_explanatory`),
-   L'ensemble le plus petit est l'ensemble vide (objet `lower_explanatory`),
-   Les variables potentielles à supprimer sont obtenues par différence (objet `remove`),

Nous initialisons l'ensemble des variables sélectionnées (objet `selected`) et réalisons notre première formule en utilisant toutes les variables sélectionnées. En utilisant `smf.ols` nous obtenons l'AIC ou le BIC du modèle de départ (`current_score`).

La boucle while commence :

-   pour chaque variable (boucle for) à supprimer, nous effectuons une régression avec l'ensemble actuel des variables moins cette variable candidate. Nous construisons une liste de triplets `score` (AIC/BIC), signe (toujours "-" car nous effectuons une sélection backward) et la variable candidate à supprimer du modèle actuel.

-   A la fin de la boucle for, nous trions toute la liste des triplets en utilisant le score et si le meilleur triplet a un `score` meilleur que `current_score` nous mettons à jour `remove`, `selected` et `current_score`, si ce n'est pas le cas, nous interrompons la boucle while.

A la fin, nous ajustons le modèle actuel et le renvoyons comme résultat.

In [4]:
def olsbackward(data, start, crit="aic", verbose=False):
    """Backward selection for linear model with smf (with formula).

    Parameters:
    -----------
    data (pandas DataFrame): DataFrame with all possible predictors
            and response
    start (string): a string giving the starting model
            (ie the starting point)
    crit (string): "aic"/"AIC" or "bic"/"BIC"
    verbose (boolean): if True verbose print

    Returns:
    --------
    model: an "optimal" linear model fitted with statsmodels
           with an intercept and
           selected by forward/backward or both algorithm with crit criterion
    """
    # criterion
    if not (crit == "aic" or crit == "AIC" or crit == "bic" or crit == "BIC"):
        raise ValueError("criterion error (should be AIC/aic or BIC/bic)")
    # starting point
    formula_start = start.split("~")
    response = formula_start[0].strip()
    # explanatory variables for the 3 models
    start_explanatory = set([item.strip() for item in
                             formula_start[1].split("+")]) - set(['1'])
    # setting up the set "remove" which contains the possible
    # variable to remove
    lower_explanatory = set([])
    remove = start_explanatory - lower_explanatory
    # current point
    selected = start_explanatory
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(list(selected)))
    if crit == "aic" or crit == "AIC":
        current_score = smf.ols(formula, data).fit().aic
    elif crit == "bic" or crit == "BIC":
        current_score = smf.ols(formula, data).fit().bic
    if verbose:
        print("----------------------------------------------")
        print((current_score, "Starting", selected))
    # main loop
    while True:
        scores_with_candidates = []
        for candidate in remove:
            tobetested = selected - set([candidate])
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(list(tobetested)))
            if crit == "aic" or crit == "AIC":
                score = smf.ols(formula, data).fit().aic
            elif crit == "bic" or crit == "BIC":
                score = smf.ols(formula, data).fit().bic
            if verbose:
                print((score, "-", candidate))
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop(0)
        if current_score > best_new_score:
            remove = remove - set([best_candidate])
            selected = selected - set([best_candidate])
            current_score = best_new_score
            if verbose:
                print("----------------------------------------------")
                print((current_score, "New Current", selected))
        else:
            break
    if verbose:
        print("----------------------------------------------")
        print((current_score, "Final", selected))
    formula = "{} ~ {} + 1".format(response, ' + '.join(list(selected)))
    model = smf.ols(formula, data).fit()
    return model

La mise en oeuvre

In [5]:
modeleaicfinal = olsbackward(ozone,"O3~T9+T12+T15+Ne9+Ne12+Ne15+Vx9+Vx12+Vx15+O3v", verbose=True)

----------------------------------------------
(925.1002020679311, 'Starting', {'Vx9', 'Vx15', 'Ne12', 'T9', 'T15', 'Ne9', 'Vx12', 'T12', 'O3v', 'Ne15'})
(924.2910760882157, '-', 'Vx9')
(923.3317000545144, '-', 'Vx15')
(923.2052359512816, '-', 'Ne12')
(923.100518756265, '-', 'T9')
(923.363923324606, '-', 'T15')
(928.9798518527307, '-', 'Ne9')
(923.1011713799451, '-', 'Vx12')
(925.7333732341065, '-', 'T12')
(953.3560377543913, '-', 'O3v')
(923.1374214147936, '-', 'Ne15')
----------------------------------------------
(923.100518756265, 'New Current', {'Vx9', 'Vx15', 'Ne12', 'T15', 'Ne9', 'Vx12', 'T12', 'O3v', 'Ne15'})
(922.5113099067607, '-', 'Vx9')
(921.3343370195821, '-', 'Vx15')
(921.2193530155344, '-', 'Ne12')
(921.3657520095587, '-', 'T15')
(927.2908775332714, '-', 'Ne9')
(921.1012966914407, '-', 'Vx12')
(924.4976674984907, '-', 'T12')
(954.0035689902516, '-', 'O3v')
(921.1374374227578, '-', 'Ne15')
----------------------------------------------
(921.1012966914407, 'New Current', {

Le modèle sélectionné

In [6]:
modeleaicfinal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     O3   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     85.75
Date:                Tue, 08 Apr 2025   Prob (F-statistic):           1.76e-32
Time:                        11:08:10   Log-Likelihood:                -451.93
No. Observations:                 112   AIC:                             913.9
Df Residuals:                     107   BIC:                             927.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.6313     11.001      1.148      0.253      -9.177      34.439
Ne9           -2.5154      0.676     -3.722      0.000      -3.855      -1.176
Vx9            1.2929      0.602      2.147      0.034       0.099       2.487
T12            2.7641      0.475      5.825      0.000       1.823       3.705
O3v            0.3548      0.058      6.130      0.000       0.240       0.470
==============================================================================
Omnibus:                        8.790   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.012   Jarque-Bera (JB):               17.762
Skew:                           0.156   Prob(JB):                     0.000139
Kurtosis:                       4.926   Cond. No.                         810.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
modelebicfinal = olsbackward(ozone,"O3~T9+T12+T15+Ne9+Ne12+Ne15+Vx9+Vx12+Vx15+O3v", crit="bic")

In [8]:
modelebicfinal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     O3   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     85.75
Date:                Tue, 08 Apr 2025   Prob (F-statistic):           1.76e-32
Time:                        11:12:29   Log-Likelihood:                -451.93
No. Observations:                 112   AIC:                             913.9
Df Residuals:                     107   BIC:                             927.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.6313     11.001      1.148      0.253      -9.177      34.439
Ne9           -2.5154      0.676     -3.722      0.000      -3.855      -1.176
Vx9            1.2929      0.602      2.147      0.034       0.099       2.487
T12            2.7641      0.475      5.825      0.000       1.823       3.705
O3v            0.3548      0.058      6.130      0.000       0.240       0.470
==============================================================================
Omnibus:                        8.790   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.012   Jarque-Bera (JB):               17.762
Skew:                           0.156   Prob(JB):                     0.000139
Kurtosis:                       4.926   Cond. No.                         810.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""